# Calculate Fixations
Status: Implemented
 -- Fully automatized

In [1]:
import warnings

import pandas as pd
from tqdm.notebook import tqdm
from utils.utils import *

warnings.filterwarnings("ignore")

In [2]:
# get all files from participants
folders = get_participant_folder_per_participant()
sequence_orders = get_sequence_order_per_participant()
assert (set(folders.keys()) == set(sequence_orders.keys()))
snippets = {p: list(sequence_orders[p][SNIPPET].values)
            for p in sequence_orders}

### Calculate fixation data
* calculate fixations

In [3]:
# Iterate over all participants
all_excluded_fixations = {}
for participant in tqdm(folders):
    print('----------------------------------------------')
    print(PARTICIPANT, participant)

    i2mc_options = I2MC_options.copy()
    i2mc_options['freq'] = get_eyetracking_frequency(participant)

    participant_snippet_gazes = {}
    participant_fixation_cross_gazes = {}
    excluded_fixations = {}
    exclusions: dict[str, dict[str, dict[str, str]]] = get_exclusions(
        participant, [SNIPPET], [BEHAVIORAL, VISUAL])

    print('(01/05) Load gaze data')
    for snippet in tqdm(snippets[participant], leave=True):
        if exclusions[SNIPPET][snippet][BEHAVIORAL] or any(s.startswith('01d1_PRE_VIS_Event_Gaze_Data') for s in exclusions[SNIPPET][snippet][VISUAL]):
            continue
        snippet_gaze_path = get_snippet_gaze_path(
            participant, snippet)
        if not snippet_gaze_path.exists():
            if not snippet in excluded_fixations:
                excluded_fixations[snippet] = '01d2_PRE_VIS_Fixation_Calculation: '
            excluded_fixations[snippet] += 'No gaze data for snippet view found.'
            continue
        fixation_cross_gaze_path = get_snippet_gaze_path(
            participant, snippet, fixation_cross=True)
        if not fixation_cross_gaze_path.exists():
            if not snippet in excluded_fixations:
                excluded_fixations[snippet] = '01d2_PRE_VIS_Fixation_Calculation: '
            excluded_fixations[snippet] += 'No gaze data for fixation cross view found.'
            continue
        participant_snippet_gazes[snippet] = pd.read_csv(
            snippet_gaze_path, index_col=False, sep=SEPARATOR, dtype={PARTICIPANT: str})
        participant_fixation_cross_gazes[snippet] = pd.read_csv(
            fixation_cross_gaze_path, index_col=False, sep=SEPARATOR, dtype={PARTICIPANT: str})

    print('(02/05) Transform gaze data to fixation data')
    # insert frequency to algorithm options before calculating fixations for each gaze data
    participant_snippet_fixations = calculate_fixations(
        participant_snippet_gazes, participant, i2mc_options)
    participant_fixation_cross_fixations = calculate_fixations(
        participant_fixation_cross_gazes, participant, i2mc_options)

    print('(03/05) Basic check of each snippet\'s fixations')
    # per snippet perform basic check
    check_fixation_cross_fixation_outliers(
        participant_snippet_fixations, participant_fixation_cross_fixations, participant)

    print('(04/05) Check fixations present for snippet')
    for snippet, fixations in tqdm(participant_snippet_fixations.items(), leave=True):
        if fixations.empty or fixations.shape[0] < 2:
            if not snippet in excluded_fixations:
                excluded_fixations[snippet] = '01d2_PRE_VIS_Fixation_Calculation: '
            excluded_fixations[
                snippet] += 'Too few fixations identified in snippet view (<2).'
    if excluded_fixations:
        update_exclusions(participant, SNIPPET, VISUAL, excluded_fixations)
    all_excluded_fixations[participant] = excluded_fixations

    print('(05/05) Store fixation data')
    # save fixation data
    for snippet in tqdm(participant_snippet_fixations):
        fixation_data = participant_snippet_fixations[snippet]
        fixation_data.to_csv(get_fixations_path(
            participant, snippet, iteration=lambda:1), index=False, sep=SEPARATOR)
    for snippet in tqdm(participant_fixation_cross_fixations):
        fixation_data = participant_fixation_cross_fixations[snippet]
        fixation_data.to_csv(get_fixations_path(
            participant, snippet, fixation_cross=True), index=False, sep=SEPARATOR)
print('Excluded trials per participant', all_excluded_fixations)
print('All trials', sum([len(trials)
      for trials in all_excluded_fixations.values()]))

  0%|          | 0/24 [00:00<?, ?it/s]

----------------------------------------------
Participant 003
(01/05) Load gaze data


  0%|          | 0/72 [00:00<?, ?it/s]

(02/05) Transform gaze data to fixation data


  0%|          | 0/69 [00:00<?, ?it/s]

...

  0%|          | 0/72 [00:00<?, ?it/s]

(05/05) Store fixation data


  0%|          | 0/72 [00:00<?, ?it/s]

  0%|          | 0/72 [00:00<?, ?it/s]

Excluded trials per participant {'003': {}, '004': {}, '005': {}, '007': {}, '008': {}, '009': {}, '010': {}, '011': {}, '012': {}, '013': {}, '014': {}, '015': {}, '016': {}, '017': {}, '018': {}, '019': {}, '020': {}, '021': {}, '022': {}, '023': {}, '024': {}, '101': {}, '102': {}, '106': {}}
All trials 0
